# La Palma Island OSM Data Case Study
---

### Map Area

La Palma, Islas Canarias, España
https://www.openstreetmap.org/relation/1464825

This is one of the Canary Islands that I do not know and I would like to know something of it before travel there.

In [16]:
%env OSMFILE=LaPalma.osm

env: OSMFILE=LaPalma.osm


First I perform a general check of elements of the map

In [15]:
%run mapparser.py

{'bounds': 1,
 'member': 8220,
 'meta': 1,
 'nd': 411918,
 'node': 359093,
 'note': 1,
 'osm': 1,
 'relation': 585,
 'tag': 86422,
 'way': 36357}


We can verify that the island is not empty at all and has about 350K nodes

## Problems Encountered in the Map
---

The main problemns I found on the osm are the followings:
+ (tags.py) Error characters, hyphen, number and some upper character in key tags 
+ (audit-streets.py) Duplicated references to same strigns, some abreviations (C/, Ctra., Avda.) and housenumber (and other fields) on street fields.  
+ (audit-phoneNumber.py) Different phone number format (spaces, ., -, 0034) and double numbers.

In [9]:
# Display problems on key tags
%run tags.py

{'lower': 69013,
 'lower_colon': 14166,
 'lower_several_colon': 3072,
 'other': 0,
 'problemchars': 0,
 'some_hyphen': 17,
 'some_number': 66,
 'some_upper': 88}


In [37]:
# Display correctons on street names
%run audit-streets.py

CALLE 30 DE MAYO Nº 1B (GENERAL TF-812 - SAN PEDRO KM 1.2) => Calle 30 DE MAYO Nº 1B (GENERAL TF-812 - SAN PEDRO KM 1.2)
CARRETERA VECINAL A PUERTO NAOS LA LAGUNA KM 3.5 => Carretera VECINAL A PUERTO NAOS LA LAGUNA KM 3.5
CARRETERA LP1- CRUCE SAN ANTONIO DEL MONTE KM. S/N => Carretera LP1- CRUCE SAN ANTONIO DEL MONTE KM. S/N
CARRETERA GENERAL LA GRAMA, KM 3.8 (CARRETERA VARIANTE TF-812 KM 3.8) => Carretera GENERAL LA GRAMA, KM 3.8 (Carretera VARIANTE TF-812 KM 3.8)
CARRETERA LP2 FUENCALIENTE KM 24.6 => Carretera LP2 FUENCALIENTE KM 24.6
Ctra. Gral. de Tajuya, 2 => Carretera Gral. de Tajuya, 2
Ctra. Gral Las Manchas => Carretera Gral Las Manchas
Ctra. La Costa-El Faro => Carretera La Costa-El Faro
Ctra. Puerto Naos A => Carretera Puerto Naos A
Ctra. General de Padrón => Carretera General de Padrón
8Ctra. Gral. Enpalme Dos Pinos => 8Carretera Gral. Enpalme Dos Pinos
Created. Gral => Carretera Gral
Crta. de Puerto Naos => Carretera de Puerto Naos
Urbanizacón La Cascade => Urbanización La 

In [34]:
# Display phone numbers corrections
%run audit-phoneNumbers.py

+34 922 48 65 69 => +34922486569
639 837 643 => +34639837643
922 48 57 35 => +34922485735
673254021 => +34673254021
+34 922 40 23 05 => +34922402305
922.482.106 / 659.135.124 => +34922482106 / +34659135124
+34 922 46 49 83 => +34922464983
+34 922 46 00 30 => +34922460030
922416304 => +34922416304
+34 922088518 => +34922088518
+34 922 49 12 30 => +34922491230
+34 922 40 04 80 => +34922400480
922464608 => +34922464608
(+34) 922-40 60 00 => +34922406000
+34 922 480443 => +34922480443
+34 922 46 38 56 => +34922463856
+34 922 444 475 => +34922444475
+34 609 531 376 => +34609531376
922 490 393 => +34922490393
922485727 => +34922485727
+34 922 464 028 => +34922464028
922 40 81 00 => +34922408100
922406146 => +34922406146
+34 619 57 11 25 => +34619571125
+34 922 40 19 13 => +34922401913
+34 922 48 06 04 => +34922480604
+34 922 493240 => +34922493240
+34 922 45 17 27 => +34922451727
+34 922 46 41 45 => +34922464145
609 909 898 => +34609909898
922 40 83 52 => +34922408352
+34 922 46 23 20 => +34

# Data Overview and Additional Ideas
---

In the following part I will perform some exploration about some useful information in order to visit the island
+ Places (Hamlets, Suburbs, Villages, ...)
+ Amenity/Shops (Restaurants, pub, cafe, supermarkets, ...)
+ Tourism (Natural and tourism points)
+ Public transport

In [67]:
from pymongo import MongoClient
import json
import pprint

client = MongoClient("mongodb://localhost:27017")
db = client.OSM

def display_results(var):
    result = db.nodes.aggregate([{"$match": { var: {"$ne": None}}},
                                 {"$group": {"_id": "$"+var, "count": {"$sum": 1}}},
                                 {"$sort": {"count": -1}},
                                 {"$limit": 10}])
    for data in result:
        pprint.pprint(data)
    client.close()    

With the results of the following query (count of place types) we can see that although the island is not very big is full enought of places.

In [76]:
display_results("place")

{'_id': 'hamlet', 'count': 87}
{'_id': 'suburb', 'count': 62}
{'_id': 'village', 'count': 30}
{'_id': 'locality', 'count': 10}
{'_id': 'isolated_dwelling', 'count': 5}
{'_id': 'island', 'count': 4}
{'_id': 'town', 'count': 3}
{'_id': 'islet', 'count': 2}
{'_id': 'square', 'count': 2}


With the results of the following query (count of amenity/shop types) we can see we have enought restaurant, bars, pubs and cafes.
Take into accound that the island is full of place with parking space.

In [68]:
display_results("amenity")

{'_id': 'parking', 'count': 446}
{'_id': 'restaurant', 'count': 181}
{'_id': 'recycling', 'count': 122}
{'_id': 'bench', 'count': 95}
{'_id': 'place_of_worship', 'count': 60}
{'_id': 'bar', 'count': 54}
{'_id': 'telephone', 'count': 51}
{'_id': 'pub', 'count': 50}
{'_id': 'toilets', 'count': 45}
{'_id': 'cafe', 'count': 44}


In [70]:
display_results("shop")

{'_id': 'supermarket', 'count': 66}
{'_id': 'clothes', 'count': 40}
{'_id': 'convenience', 'count': 19}
{'_id': 'car_repair', 'count': 12}
{'_id': 'bakery', 'count': 10}
{'_id': 'travel_agency', 'count': 9}
{'_id': 'butcher', 'count': 9}
{'_id': 'variety_store', 'count': 7}
{'_id': 'hairdresser', 'count': 7}
{'_id': 'car', 'count': 7}


The island is plenty of natural places and attraction, museum and artwork to visit during the travel.

In [71]:
display_results("natural")

{'_id': 'water', 'count': 351}
{'_id': 'tree', 'count': 224}
{'_id': 'scrub', 'count': 88}
{'_id': 'peak', 'count': 75}
{'_id': 'cliff', 'count': 63}
{'_id': 'coastline', 'count': 48}
{'_id': 'beach', 'count': 37}
{'_id': 'bare_rock', 'count': 30}
{'_id': 'spring', 'count': 27}
{'_id': 'wood', 'count': 24}


In [74]:
display_results("tourism")

{'_id': 'information', 'count': 259}
{'_id': 'viewpoint', 'count': 152}
{'_id': 'picnic_site', 'count': 51}
{'_id': 'hotel', 'count': 48}
{'_id': 'attraction', 'count': 41}
{'_id': 'chalet', 'count': 38}
{'_id': 'guest_house', 'count': 19}
{'_id': 'museum', 'count': 19}
{'_id': 'artwork', 'count': 11}
{'_id': 'camp_site', 'count': 7}


The density of platform and stops is enought to take into account use the public transport.
There are also a lot of car shops and parking spaces.
Then, it is possible use the public transport and the car renting.

In [73]:
display_results("public_transport")

{'_id': 'platform', 'count': 215}
{'_id': 'stop_position', 'count': 91}
{'_id': 'yes', 'count': 1}


# Conclusion
---

After this review I am aware of the amount of information stored in OSM and I have to say that the level of detail is high and the quality too although I have made some corrections.
The exploration has been enough to give me an idea of the characteristics of the island in terms of tourism, which is why I am satisfied with my information needs.